In [1]:
# from coco_dataloader import COCO_loader

# cl = COCO_loader()


In [2]:
from models import HW4Net1, HW4Net2, HW4Net3

import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
import os
import cv2

from tqdm import tqdm

In [3]:
# # get all images containing given categories, select one at random
# categories = ['airplane', 'bus', 'cat', 'dog', 'pizza']

# # def get_plots(image_list):
# data_path = '/mnt/cloudNAS4/akshita/Documents/datasets/coco_custom'
# num_images = 3
# all_images = []
# for category in categories:
#     cat_images = []
#     path = os.path.join(data_path, category, 'train')
#     image_list = os.listdir(path)[:num_images]
#     # print(image_list)
#     for img in image_list:
#         img = cv2.imread(os.path.join(path, img))
#         cat_images.append(img)
#     tot_images = np.concatenate(cat_images, axis=1)

#     all_images.append(tot_images)

# all_images = np.concatenate(all_images, axis=0)

# cv2.imwrite("results/sampled_images.png", all_images)

In [4]:
from dataset import MyDataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import csv

torch.manual_seed(60146)

categories = ['airplane', 'bus', 'cat', 'dog', 'pizza']
data_path = '/mnt/cloudNAS4/akshita/Documents/datasets/coco_custom'
train_dataset = MyDataset(root=data_path, categories=categories, split='train')

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


def train(net, train_dataloader, epochs=10, net_name='net_name'):
    device = 'cuda' if torch.cuda.is_available() == True else 'cpu'
    net.train()
    net = net.to(device=device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, betas=(0.9, 0.99))

    # tot_loss = []
    for epoch in tqdm(range(epochs)):
        running_loss = 0.0
        loss_flag = 1e32
        for i, data in enumerate(train_dataloader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if (i+1)% 100 == 0:
                # opening the csv file in 'w+' mode
                # print("[epoch]: %d, batch: %5d] loss: %.3f" %(epoch+1, i+1, running_loss / 100))
                data = [epoch+1, i+1, running_loss/100]
                file = open('results/'+net_name + '.csv', 'a', newline ='')
                # writing the data into the file
                with file:   
                    write = csv.writer(file)
                    write.writerow(data)
                
                # tot_loss.append(running_loss/100)
                if running_loss < loss_flag:
                    loss_flag = running_loss
                    torch.save(net.state_dict(), 'results/' + net_name + '.pt')
                running_loss = 0.0

    # return tot_loss


# torch.save(net.state_dict(), 'HW4/results/net1.pt')
# np.save('first_net.npy', np.array(loss_1), allow_pickle=True)


In [5]:
net = HW4Net1()
train(net=net, train_dataloader=train_dataloader, epochs=10, net_name='Net1')

net = HW4Net2()
train(net=net, train_dataloader=train_dataloader, epochs=10, net_name='Net2')

net = HW4Net3()
train(net, train_dataloader=train_dataloader, epochs=10, net_name='Net3')

100%|██████████| 10/10 [04:50<00:00, 29.04s/it]


$Input: (B, C_{in}, H_{in}, W_{in})$ 

$Output: (B, C_{out}, H_{out}, W_{out})$

$$H_{out} = floor\left( \frac{H_{in} + 2\times pad[0] - dilation[0] \times (kernel - 1) -1 }{stride[0]} +1 \right)$$
$$W_{out} = floor\left( \frac{W_{in} + 2\times pad[1] - dilation[1] \times (kernel - 1) -1 }{stride[1]} +1 \right)$$

In *Net1*, ```dilation``` and ```stride``` are set to default values, i.e., both 1 and ```pad = 0``` . Above equations are transformed to:

$$H_{out} = floor\left( H_{in} - kernel + 1 \right)$$
$$W_{out} = floor\left( W_{in} - kernel + 1 \right)$$

In *Net2*, ```dilation``` and ```stride``` are set to default values, i.e., both 1 and ```pad = 1``` . Above equations are transformed to:

$$H_{out} = floor\left( H_{in} + 2 - kernel + 1 \right) = floor\left( H_{in}  - kernel + 3 \right)$$
$$W_{out} = floor\left( W_{in} + 2 - kernel + 1 \right) = floor\left( W_{in}  - kernel + 3 \right)$$

